In [66]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [67]:
!pip install scikit-learn==0.24 --user

^C
ERROR: Operation cancelled by user


In [68]:
import sklearn
sklearn.__version__

'0.24.0'

In [69]:
import pandas as pd
import numpy as np

In [70]:
df = pd.read_csv('https://raw.githubusercontent.com/plenoi/Clinic/master/ultima_all_clean.csv')
df = df.set_index('hn')
df.head()

,age,parity,hiv,menopaus,disease,surgery,conization,OPDsize,appearance,stage,...,nodeyiel,RHlvsi,depth,size,utmet,vgmargin,vgmet,pelvicme,pmmet,adnmet
hn,,,,,,,,,,,,,,,,,,,,,
2631840,52,3,0.0,0.0,1,1,0.0,5.0,4.0,5.0,...,21.0,0.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
2633481,32,2,0.0,0.0,0,1,0.0,5.0,1.0,5.0,...,11.0,0.0,3.0,5.0,0.0,0.0,0.0,1.0,0.0,2.0
2634477,52,2,0.0,0.0,0,0,0.0,5.0,NaN,5.0,...,35.0,6.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2633633,38,2,0.0,0.0,0,0,0.0,2.0,1.0,4.0,...,20.0,16.0,3.0,3.8,0.0,0.0,0.0,0.0,0.0,2.0
2630496,55,3,0.0,1.0,0,0,1.0,0.0,5.0,4.0,...,17.0,9.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [71]:
df.isnull().sum()

age             0
parity          0
hiv             4
menopaus        1
disease         0
surgery         0
conization      5
OPDsize        17
appearance    101
stage          24
pchemo          1
Wardsize      145
finalhisto     10
nodeyiel       12
RHlvsi        366
depth         489
size          114
utmet          98
vgmargin       96
vgmet          97
pelvicme        1
pmmet          94
adnmet          7
dtype: int64

In [72]:
df_clean_column = df.drop(['appearance','Wardsize','RHlvsi','depth','nodeyiel','vgmargin','pelvicme','adnmet','size'],axis = 1)
df_clean_column.isnull().sum(axis=0) 

age            0
parity         0
hiv            4
menopaus       1
disease        0
surgery        0
conization     5
OPDsize       17
stage         24
pchemo         1
finalhisto    10
utmet         98
vgmet         97
pmmet         94
dtype: int64

In [73]:
df_clean = df_clean_column.dropna(axis = 0)
df_clean

,age,parity,hiv,menopaus,disease,surgery,conization,OPDsize,stage,pchemo,finalhisto,utmet,vgmet,pmmet
hn,,,,,,,,,,,,,,
2631840,52,3,0.0,0.0,1,1,0.0,5.0,5.0,0.0,3.0,0.0,0.0,0.0
2633481,32,2,0.0,0.0,0,1,0.0,5.0,5.0,0.0,1.0,0.0,0.0,0.0
2634477,52,2,0.0,0.0,0,0,0.0,5.0,5.0,0.0,4.0,0.0,0.0,0.0
2633633,38,2,0.0,0.0,0,0,0.0,2.0,4.0,0.0,3.0,0.0,0.0,0.0
2630496,55,3,0.0,1.0,0,0,1.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3169688,47,2,0.0,0.0,0,0,1.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0
3167041,39,1,0.0,0.0,0,0,1.0,0.0,2.0,0.0,5.0,0.0,0.0,0.0
2623351,46,2,0.0,0.0,0,0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [74]:
y = df_clean['pmmet'].values
X = df_clean.drop(['pmmet'],1).values 

In [75]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [76]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

# Hold Out

In [77]:
from sklearn.model_selection import train_test_split #เอาไว้แบ่งข้อมูล 2 แบบ #Holdout
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=0) 

In [78]:
from sklearn.preprocessing import MinMaxScaler #ปรับสเกล #normalizationให้ข้อมูลอยู่ช่วง 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train) #ไม่ฟิต x_test เพราะเหมือนเป็นการแอบดูเฉลย
X_train_norm = scaler.transform(X_train)
X_val_norm = scaler.transform(X_val)

In [79]:
from sklearn.linear_model import LogisticRegression #ไม่ต้องทำ gridsearch
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_val_norm)
acc = sum(yp == y_val)/len(y_val)
print("Hold Out Training Accuracy: " + str(acc))

Hold Out Training Accuracy: 0.8174603174603174


# Cross Validation

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [81]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train) 
X_train_norm = scaler.transform(X_train)

In [82]:
from sklearn.model_selection import cross_val_score
clf = LogisticRegression(random_state=0, solver='liblinear')
acc = cross_val_score(clf,X_train_norm,y_train, cv=10)
print("10CV Training Accuracy: "+str(acc.mean()))

10CV Training Accuracy: 0.8137079365079366


# PCA

In [83]:
from sklearn.decomposition import PCA #Before Normalization 
pca = PCA(n_components = 5) #ทำต่อเมื่อรู้สึกว่า data มีจำนวนฟีสเจอร์เยอะเกินแล้วอยากลด #หรือไม่ต้องทำ 5555 
pca.fit(X_train)  #เพราะเราจะไม่รู้ว่าฟีสเจอร์ตัวไหนทำให้ค่ามันสูงขึ้น
X_train_pca = pca.transform(X_train)
X_test_pca =  pca.transform(X_test)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaler.fit(X_train_pca)
X_train_norm = scaler.transform(X_train_pca)
X_test_norm = scaler.transform(X_test_pca)

In [84]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_test)/len(y_test)
print("Test Trainning Accuracy: "+str(acc))

Test Trainning Accuracy: 0.8507936507936508


# TSNE

In [85]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components = 2)
tsne.fit(X_train)
X_train_tsne = tsne.fit_transform(X_train)
X_test_tsne = tsne.fit_transform(X_test)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train_tsne)
X_train_norm = scaler.transform(X_train_tsne)
X_test_norm = scaler.transform(X_test_tsne)

In [86]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy : "+str(acc))

Test accuracy : 0.8507936507936508


# Feature Selection

**Filter Medthod** ดูฟีสเจอทีละตัว

In [87]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
fs = SelectKBest(f_classif, k=5)
fs.fit(X_train, y_train)
X_train_fs = fs.transform(X_train)
X_test_fs = fs.transform(X_test)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train_fs)
X_train_norm = scaler.transform(X_train_fs)
X_test_norm = scaler.transform(X_test_fs)

In [88]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_test)/len(y_test)
print("Test Training accuracy : "+str(acc))

Test Training accuracy : 0.8603174603174604


# Pipeline

In [89]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X_train, y_train)

params = {'C' : [1, 2, 4, 8, 16]}

from sklearn.pipeline import Pipeline
clf = Pipeline([
  ('scaler', MinMaxScaler(feature_range=(0, 1))),
  ('feature_selection', SelectKBest(f_classif, k=5)),
  ('classification', GridSearchCV(LogisticRegression(random_state=0, solver='liblinear'),params, cv=5))
])
clf.fit(X_res, y_res)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('feature_selection', SelectKBest(k=5)),
                ('classification',
                 GridSearchCV(cv=5,
                              estimator=LogisticRegression(random_state=0,
                                                           solver='liblinear'),
                              param_grid={'C': [1, 2, 4, 8, 16]}))])

In [90]:
from sklearn.model_selection import cross_val_score
acc = cross_val_score(clf, X_res, y_res, cv=10)
print("10CV Training Accuracy : "+str(acc.mean()))

10CV Training Accuracy : 0.6769440459110474


In [91]:
yp = clf.predict(X_test)
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy : "+str(acc))

Test accuracy : 0.765079365079365


# Assignment

In [92]:
!pip install xgboost

In [95]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X_train, y_train)

params = {
    'C' : [1,2,4,8,16,32], # High C = Overfitting
    'gamma' : [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32] # High gamma = Overfitting
}

from sklearn.pipeline import Pipeline
clf = Pipeline([
  ('scaler', MinMaxScaler(feature_range=(0, 1))),
  ('feature_selection', SelectKBest(f_classif, k=5)),
  ('classification', GridSearchCV(SVC(random_state=0),params, cv = 10))
])
clf.fit(X_res, y_res)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('feature_selection', SelectKBest(k=5)),
                ('classification',
                 GridSearchCV(cv=10, estimator=SVC(random_state=0),
                              param_grid={'C': [1, 2, 4, 8, 16, 32],
                                          'gamma': [0.03125, 0.0625, 0.125,
                                                    0.25, 0.5, 1, 2, 4, 8, 16,
                                                    32]}))])

In [96]:
from sklearn.model_selection import cross_val_score
acc = cross_val_score(clf, X_res, y_res, cv=10)
print("10CV Training Accuracy : "+str(acc.mean()))

10CV Training Accuracy : 0.7757361798556861


In [97]:
yp = clf.predict(X_test)
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy : "+str(acc))

Test accuracy : 0.8666666666666667
